In [15]:
import numpy as np
import open3d as o3d

# 1. 정사각형 범위 내에 랜덤 점 생성
num_points = 10000  # 원하는 점 개수
x = np.random.uniform(-10, 10, num_points)
y = np.random.uniform(-10, 10, num_points)
z = np.zeros(num_points)  # 완벽한 z=0 평면

points = np.stack((x, y, z), axis=1)

# 2. 랜덤 회전 행렬 생성
def random_rotation_matrix():
    theta = np.random.uniform(0, 2*np.pi)
    phi = np.random.uniform(0, 2*np.pi)
    z_angle = np.random.uniform(0, 2*np.pi)

    Rz = np.array([
        [np.cos(z_angle), -np.sin(z_angle), 0],
        [np.sin(z_angle),  np.cos(z_angle), 0],
        [0, 0, 1]
    ])

    Ry = np.array([
        [np.cos(phi), 0, np.sin(phi)],
        [0, 1, 0],
        [-np.sin(phi), 0, np.cos(phi)]
    ])

    Rx = np.array([
        [1, 0, 0],
        [0, np.cos(theta), -np.sin(theta)],
        [0, np.sin(theta),  np.cos(theta)]
    ])

    return Rz @ Ry @ Rx

R = random_rotation_matrix()

# 3. 점군에 회전 적용
rotated_points = (R @ points.T).T

# 4. Open3D 포인트 클라우드로 변환
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(rotated_points)

# (선택) 시각화할 수도 있음
# o3d.visualization.draw_geometries([pcd])

# 다음 블럭에서 이어서 사용할 준비 완료!


In [16]:
# 1. 회전된 포인트를 다시 가져오기
xyz_data = np.asarray(pcd.points)

# 2. 평면 추정
plane_model, inliers = pcd.segment_plane(distance_threshold=0.01,
                                         ransac_n=3,
                                         num_iterations=1000)
a, b, c, d = plane_model
print(f"추정된 평면 방정식: {a:.4f}x + {b:.4f}y + {c:.4f}z + {d:.4f} = 0")

# 3. 회전 행렬 계산 (z축 정렬용)
normal_vector = np.array([a, b, c])
normal_vector /= np.linalg.norm(normal_vector)
target_normal = np.array([0.0, 0.0, 1.0])

v = np.cross(normal_vector, target_normal)
s = np.linalg.norm(v)
c = np.dot(normal_vector, target_normal)

vx = np.array([
    [0, -v[2], v[1]],
    [v[2], 0, -v[0]],
    [-v[1], v[0], 0]
])

if s == 0:
    R_align = np.eye(3)
else:
    R_align = np.eye(3) + vx + vx @ vx * ((1 - c) / (s ** 2))

# 4. 회전 적용
pcd.rotate(R_align, center=(0, 0, 0))
rotated_points = np.asarray(pcd.points)

# 5. 회전 후 평면 추정
plane_model_rotated, _ = pcd.segment_plane(distance_threshold=0.01,
                                           ransac_n=3,
                                           num_iterations=1000)
a_rot, b_rot, c_rot, d_rot = plane_model_rotated
print(f"\n[회전 후 평면 방정식] {a_rot:.4f}x + {b_rot:.4f}y + {c_rot:.4f}z + {d_rot:.4f} = 0")

# 6. 편차 계산
distances_rotated = np.abs((a_rot * rotated_points[:,0] + b_rot * rotated_points[:,1] + c_rot * rotated_points[:,2] + d_rot) / np.sqrt(a_rot**2 + b_rot**2 + c_rot**2))

# 7. 이상치 필터링 (1m 이상 튀는 점 제거)
threshold = 1.0
inlier_mask = distances_rotated < threshold
filtered_distances = distances_rotated[inlier_mask]

# 8. 통계 출력
mean_rot = np.mean(filtered_distances)
max_rot = np.max(filtered_distances)
std_rot = np.std(filtered_distances)

print("\n✅ [회전 후 + 이상치 제거] 평탄도 분석 결과")
print(f"평균 편차 (mean): {mean_rot:.10f} m")
print(f"최대 편차 (max): {max_rot:.10f} m")
print(f"표준편차 (std): {std_rot:.10f} m")


추정된 평면 방정식: 0.9129x + -0.2585y + -0.3158z + 0.0000 = 0

[회전 후 평면 방정식] -0.0000x + 0.0000y + 1.0000z + 0.0000 = 0

✅ [회전 후 + 이상치 제거] 평탄도 분석 결과
평균 편차 (mean): 0.0000000000 m
최대 편차 (max): 0.0000000000 m
표준편차 (std): 0.0000000000 m


In [17]:
import numpy as np
import open3d as o3d

# 1. XYZ 파일 불러오기
xyz_data = np.loadtxt("flat.xyz")  # 파일명 수정

if xyz_data.shape[1] > 3:
    xyz_data = xyz_data[:, :3]
xyz_data = xyz_data.astype(np.float32)

# 2. open3d 포인트 클라우드 생성
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(xyz_data)

# 3. 평면 추정
plane_model, inliers = pcd.segment_plane(distance_threshold=0.01,
                                         ransac_n=3,
                                         num_iterations=1000)
a, b, c, d = plane_model
print(f"추정된 평면 방정식: {a:.4f}x + {b:.4f}y + {c:.4f}z + {d:.4f} = 0")

# 4. 회전 행렬 계산
normal_vector = np.array([a, b, c])
normal_vector /= np.linalg.norm(normal_vector)  # 정규화
target_normal = np.array([0.0, 0.0, 1.0])

v = np.cross(normal_vector, target_normal)
s = np.linalg.norm(v)
c = np.dot(normal_vector, target_normal)

vx = np.array([
    [0, -v[2], v[1]],
    [v[2], 0, -v[0]],
    [-v[1], v[0], 0]
])

if s == 0:
    R = np.eye(3)
else:
    R = np.eye(3) + vx + vx @ vx * ((1 - c) / (s ** 2))

# 5. 점군 회전 적용
pcd.rotate(R, center=(0, 0, 0))

# 6. 회전된 포인트 다시 배열로 변환
rotated_points = np.asarray(pcd.points)

# 7. 다시 평면 추정 + 편차 계산
plane_model_rotated, _ = pcd.segment_plane(distance_threshold=0.01,
                                           ransac_n=3,
                                           num_iterations=1000)
a_rot, b_rot, c_rot, d_rot = plane_model_rotated
print(f"\n[회전 후 평면 방정식] {a_rot:.4f}x + {b_rot:.4f}y + {c_rot:.4f}z + {d_rot:.4f} = 0")

# 편차 계산
distances_rotated = np.abs((a_rot * rotated_points[:,0] + b_rot * rotated_points[:,1] + c_rot * rotated_points[:,2] + d_rot) / np.sqrt(a_rot**2 + b_rot**2 + c_rot**2))

# ✅ [추가] 이상치 필터링 (1m 이상 튀는 점 제거)
threshold = 0.2  # 1m
inlier_mask = distances_rotated < threshold
filtered_distances = distances_rotated[inlier_mask]

# 8. 통계 다시 계산
mean_rot = np.mean(filtered_distances)
max_rot = np.max(filtered_distances)
std_rot = np.std(filtered_distances)

print("\n✅ [회전 후 + 이상치 제거] 평탄도 분석 결과")
print(f"평균 편차 (mean): {mean_rot:.6f} m")
print(f"최대 편차 (max): {max_rot:.6f} m")
print(f"표준편차 (std): {std_rot:.6f} m")


추정된 평면 방정식: 0.0512x + 0.1513y + 0.9872z + 307.9951 = 0

[회전 후 평면 방정식] 0.0036x + 0.0009y + 1.0000z + 307.8051 = 0

✅ [회전 후 + 이상치 제거] 평탄도 분석 결과
평균 편차 (mean): 0.083759 m
최대 편차 (max): 0.199126 m
표준편차 (std): 0.056154 m
